In [18]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

1. Загрузить набор данных MNIST, который включает в себя рукописные цифры от 0 до 9.
2. Предобработать данные, чтобы они были приведены к нужному формату и масштабу. 

In [19]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,)),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

3. Создать модель нейронной сети с использованием PyTorch. Модель должна содержать несколько слоев, включая скрытые слои.


In [20]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28) 
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x)) 
        x = self.fc3(x)  
        return x

model = Network()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

4. Обучить модель на тренировочном наборе данных, используя функцию потерь и оптимизатор из PyTorch.

In [21]:
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
print('Обучение модели завершено')

Epoch [1/10], Loss: 0.39786770939826965
Epoch [2/10], Loss: 0.4532583951950073
Epoch [3/10], Loss: 0.15045301616191864
Epoch [4/10], Loss: 0.28639310598373413
Epoch [5/10], Loss: 0.37352362275123596
Epoch [6/10], Loss: 0.14409643411636353
Epoch [7/10], Loss: 0.3900527060031891
Epoch [8/10], Loss: 0.26890185475349426
Epoch [9/10], Loss: 0.1152222603559494
Epoch [10/10], Loss: 0.08424311131238937
Обучение модели завершено


5. Оценить качество модели на тестовом наборе данных

In [22]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

Accuracy on the test set: 94.85%


6. Изменить параметры модели (например, число скрытых слоев, количество нейронов в слоях) и сравнить их влияние на качество распознавания.

In [23]:
class ModelReLU(nn.Module):
    def __init__(self):
        super(ModelReLU, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class ModelSigmoid(nn.Module):
    def __init__(self):
        super(ModelSigmoid, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

7. Изменить функцию активации в нейронной сети (например, ReLU,
сигмоида) и сравнить их влияние на качество распознавания

In [24]:
criterion = nn.CrossEntropyLoss()

model_relu = ModelReLU()
optimizer_relu = optim.SGD(model_relu.parameters(), lr=0.01)
train_model(model_relu, train_loader, criterion, optimizer_relu)
accuracy_relu = evaluate_model(model_relu, test_loader)


model_sigmoid = ModelSigmoid()
optimizer_sigmoid = optim.SGD(model_sigmoid.parameters(), lr=0.01)
train_model(model_sigmoid, train_loader, criterion, optimizer_sigmoid)
accuracy_sigmoid = evaluate_model(model_sigmoid, test_loader)


print(f'Accuracy with ReLU: {accuracy_relu * 100:.2f}%')
print(f'Accuracy with Sigmoid: {accuracy_sigmoid * 100:.2f}%')

Accuracy with ReLU: 92.65%
Accuracy with Sigmoid: 70.66%
